## Importing packages and attributes

In [1]:
import scraping_class
import pandas as pd
import requests,os,time, re
from datetime import datetime
from time import gmtime, strftime
#import bs4 as bs
from urllib.request import urlopen as ureq
from bs4 import BeautifulSoup as soup
#import urllib.request


pd.options.display.max_colwidth = 200

## Running of connector for HTML scraper - Created by Snorre Ralund

In [2]:
def ratelimit():
    "A function that handles the rate of your calls."
    time.sleep(1) # sleep one second.

class Connector():
  def __init__(self,logfile,overwrite_log=False,connector_type='requests',session=False,path2selenium='',n_tries = 5,timeout=30):
    """This Class implements a method for reliable connection to the internet and monitoring.
    It handles simple errors due to connection problems, and logs a range of information for basic quality assessment
    
    Keyword arguments:
    logfile -- path to the logfile
    overwrite_log -- bool, defining if logfile should be cleared (rarely the case).
    connector_type -- use the 'requests' module or the 'selenium'. Will have different since the selenium webdriver does not have a similar response object when using the get method, and monitoring the behavior cannot be automated in the same way.
    session -- requests.session object. For defining custom headers and proxies.
    path2selenium -- str, sets the path to the geckodriver needed when using selenium.
    n_tries -- int, defines the number of retries the *get* method will try to avoid random connection errors.
    timeout -- int, seconds the get request will wait for the server to respond, again to avoid connection errors.
    """
    
    ## Initialization function defining parameters. 
    self.n_tries = n_tries # For avoiding triviel error e.g. connection errors, this defines how many times it will retry.
    self.timeout = timeout # Defining the maximum time to wait for a server to response.
    ## not implemented here, if you use selenium.
    if connector_type=='selenium':
      assert path2selenium!='', "You need to specify the path to you geckodriver if you want to use Selenium"
      from selenium import webdriver 
      ## HIN download the latest geckodriver here: https://github.com/mozilla/geckodriver/releases

      assert os.path.isfile(path2selenium),'You need to insert a valid path2selenium the path to your geckodriver. You can download the latest geckodriver here: https://github.com/mozilla/geckodriver/releases'
      self.browser = webdriver.Firefox(executable_path=path2selenium) # start the browser with a path to the geckodriver.

    self.connector_type = connector_type # set the connector_type

    if session: # set the custom session
      self.session = session
    else:
      self.session = requests.session()
    self.logfilename = logfile # set the logfile path
    ## define header for the logfile\n",
    header = ['id','project','connector_type','t', 'delta_t', 'url', 'redirect_url','response_size', 'response_code','success','error']
    if os.path.isfile(logfile):        
      if overwrite_log==True:
        self.log = open(logfile,'w')
        self.log.write(';'.join(header))
      else:
        self.log = open(logfile,'a')
    else:
      self.log = open(logfile,'w')
      self.log.write(';'.join(header))
    ## load log 
    with open(logfile,'r') as f: # open file
        
      l = f.read().split('\\n') # read and split file by newlines.
      ## set id
      if len(l)<=1:
        self.id = 0
      else:
        self.id = int(l[-1][0])+1
           
  def get(self,url,project_name):
    """Method for connector reliably to the internet, with multiple tries and simple error handling, as well as default logging function.
    Input url and the project name for the log (i.e. is it part of mapping the domain, or is it the part of the final stage in the data collection).
  
    Keyword arguments:
    url -- str, url
    project_name -- str, Name used for analyzing the log. Use case could be the 'Mapping of domain','Meta_data_collection','main data collection'. 
    """
 
    project_name = project_name.replace(';','-') # make sure the default csv seperator is not in the project_name.
    if self.connector_type=='requests': # Determine connector method.
      for _ in range(self.n_tries): # for loop defining number of retries with the requests method.
        ratelimit()
        t = time.time()
        try: # error handling 
          response = self.session.get(url,timeout = self.timeout) # make get call

          err = '' # define python error variable as empty assumming success.
          success = True # define success variable
          redirect_url = response.url # log current url, after potential redirects 
          dt = t - time.time() # define delta-time waiting for the server and downloading content.
          size = len(response.text) # define variable for size of html content of the response.
          response_code = response.status_code # log status code.
          ## log...
          call_id = self.id # get current unique identifier for the call
          self.id+=1 # increment call id
          #['id','project_name','connector_type','t', 'delta_t', 'url', 'redirect_url','response_size', 'response_code','success','error']
          row = [call_id,project_name,self.connector_type,t,dt,url,redirect_url,size,response_code,success,err] # define row to be written in the log.
          self.log.write('\\n'+';'.join(map(str,row))) # write log.
          return response,call_id # return response and unique identifier.

        except Exception as e: # define error condition
          err = str(e) # python error
          response_code = '' # blank response code 
          success = False # call success = False
          size = 0 # content is empty.
          redirect_url = '' # redirect url empty 
          dt = t - time.time() # define delta t

          ## log...
          call_id = self.id # define unique identifier
          self.id+=1 # increment call_id

          row = [call_id,project_name,self.connector_type,t,dt,url,redirect_url,size,response_code,success,err] # define row
          self.log.write('\\n'+';'.join(map(str,row))) # write row to log.
    else:
      t = time.time()
      ratelimit()
      self.browser.get(url) # use selenium get method
      ## log
      call_id = self.id # define unique identifier for the call.
      self.id+=1 # increment the call_id
      err = '' # blank error message
      success = '' # success blank
      redirect_url = self.browser.current_url # redirect url.
      dt = t - time.time() # get time for get method ... NOTE: not necessarily the complete load time.
      size = len(self.browser.page_source) # get size of content ... NOTE: not necessarily correct, since selenium works in the background, and could still be loading.
      response_code = '' # empty response code.
      row = [call_id,project_name,self.connector_type,t,dt,url,redirect_url,size,response_code,success,err] # define row 
      self.log.write('\\n'+';'.join(map(str,row))) # write row to log file.
    # Using selenium it will not return a response object, instead you should call the browser object of the connector.
    ## connector.browser.page_source will give you the html.
      return call_id

logfile="tripadvisor_scraper.txt" # name your log file.
connector = Connector(logfile)

## HTML scraper - 1 step
The below code gets an overview of all the restaurant in the Copenhagen area

In [3]:
url_init = 'https://www.tripadvisor.dk' # TripAdvisor base url
max_restaurant = 2324 # Restaurants stated on he webpage, to be located in Copenhangen area. If this isn't given, the loop will continue
init_number = 0 
final_urls = []
year = datetime.now().year
month = datetime.now().month
day = datetime.now().day


while init_number <= max_restaurant:
    timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S') # Creates a timestamp in the format yyyy-mm-dd h:m:s
    
    url_search = '/RestaurantSearch-g189541-o' # first part of url
    url_search2 = '-a_date.' + str(year) + '__2D__' + str(month) + '__2D__' + str(day) + '-a_people.2-a_time.20%3A00%3A00-a_zur.' + str(year) + '__5F__' + str(month) + '__5F__' + str(day) + '-Copenhag.html' # second part of url
    url_search3 = url_init + url_search + str(init_number) + url_search2 # combined dynamic url
    data, call_id = connector.get(url_search3, 'exam_init') # gathering data
    response = data.ok # checks if datastream is possible
    final_urls.append(url_search3)
    if response == True:
        status = 'Ok'

    print('Status:', status , 'Timestamp:', timestamp, 'links processed:', len(final_urls)) # prints response, timestamp and amount of links processed.
    init_number += 30 # incriments the number by 30 to be put into the url on next call."

Status: Ok Timestamp: 2019-08-28 13:03:50 links processed: 1
Status: Ok Timestamp: 2019-08-28 13:03:52 links processed: 2
Status: Ok Timestamp: 2019-08-28 13:03:54 links processed: 3
Status: Ok Timestamp: 2019-08-28 13:03:56 links processed: 4
Status: Ok Timestamp: 2019-08-28 13:03:59 links processed: 5
Status: Ok Timestamp: 2019-08-28 13:04:01 links processed: 6
Status: Ok Timestamp: 2019-08-28 13:04:03 links processed: 7
Status: Ok Timestamp: 2019-08-28 13:04:05 links processed: 8
Status: Ok Timestamp: 2019-08-28 13:04:07 links processed: 9
Status: Ok Timestamp: 2019-08-28 13:04:09 links processed: 10
Status: Ok Timestamp: 2019-08-28 13:04:11 links processed: 11
Status: Ok Timestamp: 2019-08-28 13:04:13 links processed: 12
Status: Ok Timestamp: 2019-08-28 13:04:15 links processed: 13
Status: Ok Timestamp: 2019-08-28 13:04:17 links processed: 14
Status: Ok Timestamp: 2019-08-28 13:04:19 links processed: 15
Status: Ok Timestamp: 2019-08-28 13:04:21 links processed: 16
Status: Ok Timest

### Saving to CSV file

In [4]:
overview_urls = pd.DataFrame(final_urls)
overview_urls.columns = ['Init_links']
overview_urls.to_csv('overview_urls.csv', index = None, header = True) # writes data from df to csv file a

## HTML scraper - 2 step
The code below takes the links generated in step 1 as input, and gives us the individual links to all the restaurants in the Copenhagen area.

In [5]:
final_urls = pd.read_csv("overview_urls.csv")

In [6]:
page_list = []

for url in final_urls['Init_links'][:]: # loops over all overview sites
    response,call_id = connector.get(url,'exam_links')
    link_locations = response.text.split('href="')[1:] # find all links on site 
    link_list = [] 
    
    for i in link_locations:
        if "Restaurant_Review" in i: # finds all links named something with Restaurant review
            link_list.append(i.partition(" ")[0])

        review_list=  []
    for i in link_list:
        if "Restaurant_Review" in i and "#REVIEWS" not in i and "button" not in i: # same links appart from this removes two alternative versions and only stores one.
            review_list.append(i)

            review_list_u = set(review_list) # remove dublicates

    for i in review_list_u:
        timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S') # Creates a timestamp in the format yyyy-mm-dd h:m:s
        page_list.append(url_init+i) # make list of final urls
        if response == True:
            status = 'Ok'
        
        print('Status:', status , 'Timestamp:', timestamp, 'links processed:', len(page_list)) # prints response, timestamp and amount of links processed.   

Status: Ok Timestamp: 2019-08-28 13:07:03 links processed: 1
Status: Ok Timestamp: 2019-08-28 13:07:03 links processed: 2
Status: Ok Timestamp: 2019-08-28 13:07:03 links processed: 3
Status: Ok Timestamp: 2019-08-28 13:07:03 links processed: 4
Status: Ok Timestamp: 2019-08-28 13:07:03 links processed: 5
Status: Ok Timestamp: 2019-08-28 13:07:03 links processed: 6
Status: Ok Timestamp: 2019-08-28 13:07:03 links processed: 7
Status: Ok Timestamp: 2019-08-28 13:07:03 links processed: 8
Status: Ok Timestamp: 2019-08-28 13:07:03 links processed: 9
Status: Ok Timestamp: 2019-08-28 13:07:03 links processed: 10
Status: Ok Timestamp: 2019-08-28 13:07:03 links processed: 11
Status: Ok Timestamp: 2019-08-28 13:07:03 links processed: 12
Status: Ok Timestamp: 2019-08-28 13:07:03 links processed: 13
Status: Ok Timestamp: 2019-08-28 13:07:03 links processed: 14
Status: Ok Timestamp: 2019-08-28 13:07:03 links processed: 15
Status: Ok Timestamp: 2019-08-28 13:07:03 links processed: 16
Status: Ok Timest

In [7]:
indvidual_urls = pd.DataFrame(page_list)

In [8]:
indvidual_urls.columns = ['Restaurant_links']
indvidual_urls.to_csv('indvidual_urls.csv', index = None, header = True) # writes df to csv file

### HTML scraper - step 3
The below code takes the links for step 2 as input and retreives all the data that we ask for, for the individual restaurants.

In [3]:
# loading urls from csv
ind_urls = pd.read_csv("indvidual_urls_keep.csv")
ratings_df = pd.DataFrame()


# Creating empty lists to store data in
loc_list = []
reviewCount_list = []
distance_list = []
unique_list = []
price_class_list = []
main_rating_list = []
ranking_list = []
price_class_value_list = []
type_food_list = []
extented_adress_list = []

In [4]:
# loading data for individual restaurants.
for url in ind_urls["Restaurant_links"][0:100]:

    # loads the url in the loop and loads the html code
    trip = ureq(url)
    trip_html = trip.read()
    trip.close()
    trip_soup = soup(trip_html, "lxml")
    
    # checks if there is data in the soup.
    if len(trip_soup.text) > 0:
        status = 'Ok'
    else:
        status = 'Failed'
        print(status)
    
    # Exstracting restaurant names and detailed ratings
    name = trip_soup.findAll(True, {"class":["ui_header h1"]})
    name = str(name)
    name = name[26:]
    name = name.replace("</h1>]", '')
    small_bub = trip_soup.findAll(True, {"class":["restaurants-detail-overview-cards-RatingsOverviewCard__ratingText--1P1Lq", "restaurants-detail-overview-cards-RatingsOverviewCard__ratingBubbles--1kQYC"]})

    
    small_ratings = []
    for x in small_bub:
        small_ratings.append(str(x))
    
    keys = small_ratings[0::2]
    values = small_ratings[1::2]
    keys.append("Restaurant")
    values.append(str(name))

    keys[:] = [s.replace('<span class="restaurants-detail-overview-cards-RatingsOverviewCard__ratingText--1P1Lq">', '') for s in keys]
    keys[:] = [s.replace('</span>', '') for s in keys]
    values[:] = [s.replace('<span class="restaurants-detail-overview-cards-RatingsOverviewCard__ratingBubbles--1kQYC"><span class="ui_bubble_rating bubble_', '') for s in values]
    values[:] = [s.replace('"></span></span>', '') for s in values]
    
    ratings_dict = {}
    
    for i in range(len(keys)):
        ratings_dict[keys[i]] = values[i]
    
    #append dict to dataframe, with names and detailed ratings
    ratings_df = ratings_df.append([ratings_dict], ignore_index=True)    
    
    #Exstracting locations with long, lat format
    p = re.compile(r'"coords":"(.*?)"')
    r = requests.get(url)
    coords = p.findall(r.text)[1]
    loc_list.append(coords)
    
    #Exstracting review counts
    try:
        reviewCount = str(trip_soup.find(class_="reviewCount"))
        reviewCount = reviewCount.split(">")[1].split("<")[0]
        reviewCount = re.sub('[^0-9]', '', reviewCount)
        reviewCount_list.append(reviewCount)
    except:
        reviewCount_list.append('NaN')
    
    #Exstracting price class as $ 
    try:
        price_class_number = str(trip_soup.find('div', class_="header_links"))
        price_class = re.sub('[^$-]', '', price_class_number)
        price_class_list.append(price_class)
    except:
        price_class_list.append('NaN')

    #Exstracting restaurant ratings
    try:
        bubbles = str(trip_soup.find(class_="restaurants-detail-overview-cards-RatingsOverviewCard__overallRating--nohTl"))    
        main_rating = re.sub('[^0-9,.]', '', bubbles) #stripping all other than the ranking numbers
        main_rating_list.append(main_rating)
    except:
        main_rating_list.append('NaN')
    
    #Exstracting restaurant ranking
    try:
        list_ranking =str(trip_soup.find(class_="restaurants-detail-overview-cards-RatingsOverviewCard__ranking--17CmN").find('span', class_=""))
        ranking = re.sub('[^0-9,]', '', list_ranking)
        ranking_list.append(ranking)
    except:
        ranking_list.append('NaN')
    
    #Exstracting price_class_value
    try:
        price_class_value = str(trip_soup.find(class_="restaurants-detail-overview-cards-DetailsSectionOverviewCard__tagText--1OH6h"))
        price_class_value = re.sub('[^0-9,.-]', '', price_class_value).split('-16')[1].strip('.').split('-')[1]
        price_class_value_list.append(price_class_value)
    except:
        price_class_value_list.append('NaN')
        
    #Exstracting type of food
    try:
        type_food = str(trip_soup.find(class_="prw_rup prw_restaurants_restaurant_detail_tags tagsContainer").find(class_="header_links"))
        #kokken_list = ['Café', 'Afrikansk','Amerikansk','Arabisk','Argentinsk','Armensk','Aserbajdsjansk','Asiatisk','Bar','Belgisk','Brasiliansk','Britisk','Cajun og kreolsk','Cambodjansk','Canadisk','Caribisk','Centralasiatisk','Centraleuropæisk','Centralitaliensk','Dansk','Delikatesseforretning','Egyptisk','Etiopisk','Europæisk','Fastfood','Filippinsk','Fisk og skaldyr','Fra Lazio','Fra Shanghai','Fransk','Fusion','Gademad','Gastropub','Grill','Grillmad','Græsk','Hawaiiansk','Hollandsk','Hongkong','Indiansk','Indisk','Indonesisk','International','Irsk','Israelsk','Italiensk','Japansk','Kantonesisk','Kinesisk','Koreansk','Kroatisk','Latinamerikansk','Libanesisk','Malaysisk','Marokkansk','Mellemamerikansk','Mellemøstlig','Mexicansk','Middelhavsområdet','Moderne','Mongolsk','Nepalesisk','New Zealand','Norditaliensk','Norsk','Pakistansk','Persisk','Peruviansk','Pizza','Portugisisk''Pub','Russisk','Schweizisk','Siciliansk','Singaporeansk','Skandinavisk','Spansk','Specialiteter fra Beijing','Spisested','Steakhouse','Sund','Supper','Sushi','Svensk','Sydamerikansk','Syditaliensk','Szechuan','Taiwansk','Thai','Tibetansk','Toscansk','Tyrkisk','Tysk','Venezuelansk','Vietnamesisk','Vinstue','Xinjiang','Ølpub','Østeuropæisk','Østrigsk']
        #for v in kokken_list: 
            #if v in type_food:
                #type_food = i , i
        type_food_list.append(type_food)
    except:
        type_food_list.append('NaN')
    
    #Exstracting address
    try:
        extented_adress = str(trip_soup.find(class_="restaurants-detail-overview-cards-LocationOverviewCard__detailLinkText--co3ei"))
        extented_adress = extented_adress.split(">")[1].split("<")[0]
        extented_adress_list.append(extented_adress)
    except:
        extented_adress_list.append('NaN')
    
    
    timestamp = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
    
    print('Status:', status , 'Timestamp:', timestamp, 'pages scraped:', len(ratings_df)) # prints response, timestamp and amount of links processed.
        
    
#out of loop
ratings_df["Location"] = loc_list
ratings_df["Number of reviews"] = reviewCount_list
ratings_df["Price class"] = price_class_list
ratings_df["Main rating"] = main_rating_list
ratings_df["Ranking on list"] = ranking_list
ratings_df["Price range"] = price_class_value_list
ratings_df["Type of food"] = type_food_list
ratings_df["Address"] = extented_adress_list

Status: Ok Timestamp: 2019-08-28 13:56:13 pages scraped: 1


/Users/michaelemcken/anaconda3/lib/python3.7/site-packages/pandas/core/indexing.py:1494: FutureWarning: 
Passing list-likes to .loc or [] with any missing label will raise
KeyError in the future, you can use .reindex() as an alternative.

See the documentation here:
https://pandas.pydata.org/pandas-docs/stable/indexing.html#deprecate-loc-reindex-listlike
  return self._getitem_tuple(key)


Status: Ok Timestamp: 2019-08-28 13:56:16 pages scraped: 2
Status: Ok Timestamp: 2019-08-28 13:56:20 pages scraped: 3
Status: Ok Timestamp: 2019-08-28 13:56:24 pages scraped: 4
Status: Ok Timestamp: 2019-08-28 13:56:28 pages scraped: 5
Status: Ok Timestamp: 2019-08-28 13:56:32 pages scraped: 6
Status: Ok Timestamp: 2019-08-28 13:56:38 pages scraped: 7
Status: Ok Timestamp: 2019-08-28 13:56:42 pages scraped: 8
Status: Ok Timestamp: 2019-08-28 13:56:45 pages scraped: 9
Status: Ok Timestamp: 2019-08-28 13:56:50 pages scraped: 10
Status: Ok Timestamp: 2019-08-28 13:56:55 pages scraped: 11
Status: Ok Timestamp: 2019-08-28 13:56:59 pages scraped: 12
Status: Ok Timestamp: 2019-08-28 13:57:03 pages scraped: 13
Status: Ok Timestamp: 2019-08-28 13:57:06 pages scraped: 14
Status: Ok Timestamp: 2019-08-28 13:57:11 pages scraped: 15
Status: Ok Timestamp: 2019-08-28 13:57:15 pages scraped: 16
Status: Ok Timestamp: 2019-08-28 13:57:19 pages scraped: 17
Status: Ok Timestamp: 2019-08-28 13:57:22 pages 

## Saving raw data to CSV

Now that the scraper has gathered all the restaurants individual data, we save this to a CSV file, so we avoid to keep scraping data from TripAdvisors website.

In [5]:
ratings_df.to_csv("Tripadvisordata_raw.csv", index=False)
ratings_df

,God pris,Mad,Restaurant,Service,Stemning,Location,Number of reviews,Price class,Main rating,Ranking on list,Price range,Type of food,Address
0,45,45,Restaurant Grønnegade,45,45,"55.681705,12.583386",525,--$$$$---,"4,5",28,664,"<div class=""header_links""><a href=""/Restaurants-g189541-Copenhagen_Zealand.html?pid=8"">$$$$</a>, <a href=""/Restaurants-g189541-c33-Copenhagen_Zealand.html"">Fisk og skaldyr</a>, Dansk, Skandinavisk...","Grønnegade 39, København 1107 Danmark"
1,45,45,Basso København,45,NaN,"55.68388,12.58657",345,--$$-$$$---------,"4,5",18,496,"<div class=""header_links""><a href=""/Restaurants-g189541-Copenhagen_Zealand.html?pid=6"">$$ - $$$</a>, <a href=""/Restaurants-g189541-c26-Copenhagen_Zealand.html"">Italiensk</a>, <a href=""/Restaurants...","Dronningens Tvaergade 22, København 1302 Danmark"
2,45,50,Enomania,50,45,"55.670773,12.531963",245,--$$-$$$------,"4,5",1,NaN,"<div class=""header_links""><a href=""/Restaurants-g806262-Frederiksberg_Copenhagen_Zealand.html?pid=6"">$$ - $$$</a>, <a href=""/Restaurants-g806262-c26-Frederiksberg_Copenhagen_Zealand.html"">Italiens...","Vesterbrogade 187, Frederiksberg, København 1800 Danmark"
3,45,50,Restaurant Krebsegaarden,50,45,"55.67872,12.569877",1311,--$$$$---,"5,0",2,396,"<div class=""header_links""><a href=""/Restaurants-g189541-Copenhagen_Zealand.html?pid=8"">$$$$</a>, <a href=""/Restaurants-g189541-c18-Copenhagen_Zealand.html"">Europæisk</a>, Skandinavisk, Dansk</div>","Studiestraede 17, København 1455 Danmark"
4,45,50,The Pescatarian,50,NaN,"55.68799,12.596316",201,--$$$$------,"5,0",6,597,"<div class=""header_links""><a href=""/Restaurants-g189541-Copenhagen_Zealand.html?pid=8"">$$$$</a>, Dansk, <a href=""/Restaurants-g189541-c33-Copenhagen_Zealand.html"">Fisk og skaldyr</a>, <a href=""/Re...","Amaliegade 49 On the corner of Amaliegade &amp; Esplanaden, København 1256 Danmark"
5,45,45,Goomi,45,45,"55.673717,12.589529",271,--$$-$$$------,"4,5",25,449,"<div class=""header_links""><a href=""/Restaurants-g189541-Copenhagen_Zealand.html?pid=6"">$$ - $$$</a>, <a href=""/Restaurants-g189541-c11-Copenhagen_Zealand.html"">Kinesisk</a>, <a href=""/Restaurants-...","Strandgade 20, København 1401 Danmark"
6,45,45,Mielcke &amp; Hurtigkarl,45,45,"55.67468,12.530938",253,--$$$$---,"4,5",2,2.615,"<div class=""header_links""><a href=""/Restaurants-g806262-Frederiksberg_Copenhagen_Zealand.html?pid=8"">$$$$</a>, <a href=""/Restaurants-g806262-c18-Frederiksberg_Copenhagen_Zealand.html"">Europæisk</a...","Fr.Berg Runddel 1, Frederiksberg, København 2000 Danmark"
7,45,45,Feed Bistro,45,NaN,"55.688663,12.559759",548,--$$-$$$---,"4,5",26,NaN,"<div class=""header_links""><a href=""/Restaurants-g189541-Copenhagen_Zealand.html?pid=6"">$$ - $$$</a>, <a href=""/Restaurants-g189541-c18-Copenhagen_Zealand.html"">Europæisk</a>, Skandinavisk, Dansk</...","Faelledvej 5, København 2200 Danmark"
8,45,50,Marv &amp; Ben,50,NaN,"55.677303,12.576991",645,--$$$$---,"4,5",10,838,"<div class=""header_links""><a href=""/Restaurants-g189541-Copenhagen_Zealand.html?pid=8"">$$$$</a>, <a href=""/Restaurants-g189541-c18-Copenhagen_Zealand.html"">Europæisk</a>, Skandinavisk, Dansk</div>","Snaregade 4, København 1205 Danmark"
9,45,45,Ché Fè,45,40,"55.684223,12.585324",581,--$$-$$$---------,"4,5",29,496,"<div class=""header_links""><a href=""/Restaurants-g189541-Copenhagen_Zealand.html?pid=6"">$$ - $$$</a>, <a href=""/Restaurants-g189541-c26-Copenhagen_Zealand.html"">Italiensk</a>, <a href=""/Restaurants...","Borgergade 17A, København 1300 Danmark"


In [215]:
#ratings_df_1 = pd.read_csv("Tripadvisordata_1000.csv")
#ratings_df_2 = pd.read_csv("Tripadvisordata_1000_1200.csv")
#ratings_df_1 = ratings_df_1.append(ratings_df_2)
#ratings_df_1.to_csv("Tripadvisordata_1200.csv", index=False)